In [20]:
import requests
import tarfile
import psycopg2
from psycopg2 import sql
import subprocess


db_params = {
    'user': 'postgres',
    'password': 'admin123',
    'host': '192.168.1.2',
    'port': '5432'
}

url_api = "https://testt.order-system.trust-growth.co.jp/api"
api_key = "uED24kJ7mAZe"



In [21]:

# Define the API URL
api_url_export = url_api + '/pg_export_sql'

# Define query parameters, including your API key
params = {
    'key_api': api_key,
}

try:
    response = requests.get(api_url_export, params=params)
    
    if response.status_code == 200:
        # API request was successful
        data = response.json()  # Parse the JSON response
        # You can now work with the 'data' variable, which contains the response from the API
        print(data)
    else:
        print(f"API request failed with status code: {response.status_code}")
except Exception as e:
    print(f"An error occurred: {e}")

{'path': 'https://testt.order-system.trust-growth.co.jp/aws_dev_odsystem_20231017_114802.dump.tar.gz', 'file_name': 'aws_dev_odsystem_20231017_114802.dump', 'status_dump': 0, 'status': True}


In [22]:
data['path']
new_database = data['file_name']

'https://testt.order-system.trust-growth.co.jp/aws_dev_odsystem_20231017_114802.dump.tar.gz'

In [23]:
url = data['path']
output_file = data['file_name'] + ".tar.gz"

try:
    response = requests.get(url)
    if response.status_code == 200:
        with open(output_file, 'wb') as file:
            file.write(response.content)
        print(f"File downloaded as {output_file}")

        input_file = output_file
        output_dir = "./"

        try:
            with tarfile.open(input_file, "r:gz") as tar:
                tar.extractall(output_dir)
            print(f"File '{input_file}' has been successfully extracted to '{output_dir}'")
        except Exception as e:
            print(f"An error occurred: {str(e)}")
            
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
except Exception as e:
    print(f"An error occurred: {str(e)}")


File downloaded as aws_dev_odsystem_20231017_114802.dump.tar.gz
File 'aws_dev_odsystem_20231017_114802.dump.tar.gz' has been successfully extracted to './'


In [24]:
def create_database(new_db_name):
    try:
        conn = psycopg2.connect(**db_params)
        conn.autocommit = True
        cursor = conn.cursor()

        # Check if the database exists before creating it
        cursor.execute(sql.SQL("SELECT 1 FROM pg_database WHERE datname = {}").format(sql.Literal(new_db_name)))
        if not cursor.fetchone():
            cursor.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier(new_db_name)))
            print(f"Database '{new_db_name}' created successfully.")
        else:
            print(f"Database '{new_db_name}' already exists.")

        cursor.close()
        conn.close()
    except Exception as e:
        print(f"Error creating the database: {str(e)}")

def import_sql_file(sql_file, new_db_name, db_params):
    try:
        pg_restore_path = r'C:\Program Files\PostgreSQL\14\bin\pg_restore' 
        process = subprocess.Popen(
            [
                pg_restore_path,  # Assuming that 'pg_restore' is in your system's PATH
                f'--dbname=postgresql://{db_params["user"]}:{db_params["password"]}@{db_params["host"]}:5432/{new_db_name}',
                sql_file
            ],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE
        )
        stdout, stderr = process.communicate()
        print("Standard Output:")
        print(stdout.decode("utf-8"))
        print("Standard Error:")
        print(stderr.decode("utf-8"))

        if process.returncode == 0:
            print(f'Successfully imported SQL file into {new_db_name}')
        else:
            print(f'Error importing SQL file. Return code: {process.returncode}')
    except Exception as e:
        print(f'Issue with the db restore: {e}')


if __name__ == "__main__":
    create_database(data['file_name'])  # Create the new database
    import_sql_file(data['file_name'], data['file_name'],db_params)  # Import your SQL file


Database 'aws_dev_odsystem_20231017_114802.dump' created successfully.
Standard Output:

Standard Error:
pg_restore: while PROCESSING TOC:
pg_restore: from TOC entry 354; 1255 18438 FUNCTION convert_lang_japanese(text) aws_pg_test_odsystem
pg_restore: error: could not execute query: ERROR:  role "aws_pg_test_odsystem" does not exist
Command was: ALTER FUNCTION public.convert_lang_japanese(p_src text) OWNER TO aws_pg_test_odsystem;

pg_restore: from TOC entry 355; 1255 18439 FUNCTION import_material_supplier(integer, character varying) aws_pg_test_odsystem
pg_restore: error: could not execute query: ERROR:  role "aws_pg_test_odsystem" does not exist
Command was: ALTER FUNCTION public.import_material_supplier(auth_id integer, _token character varying) OWNER TO aws_pg_test_odsystem;

pg_restore: from TOC entry 356; 1255 18440 FUNCTION import_supplier(integer, character varying) aws_pg_test_odsystem
pg_restore: error: could not execute query: ERROR:  role "aws_pg_test_odsystem" does not ex

In [25]:
from pathlib import Path

api_url = url_api + "/unlink_file"

file_name = data['file_name'] + ".tar.gz"


file_sql = Path(data['file_name'])
file_tar = Path(file_name)

# Define the payload as a dictionary
payload = {
    "key_api": api_key,
    "file_name": file_name
}

# Make the POST request
response = requests.post(api_url, data=payload)

# Check the response status code
if response.status_code == 200:
    print("POST request successful.")
    if file_sql.exists():
        # Delete the file
        file_sql.unlink()
        print(f"File {file_sql} has been deleted.")
    if file_tar.exists():
        # Delete the file
        file_tar.unlink()
        print(f"File {file_tar} has been deleted.")
    # You can access the response content using response.text
else:
    print(f"POST request failed with status code: {response.status_code}")
    print(response.text)  # Display the response content in case of an error

POST request successful.
File aws_dev_odsystem_20231017_114802.dump has been deleted.
File aws_dev_odsystem_20231017_114802.dump.tar.gz has been deleted.


In [ ]:
# reset suppliers_send_setting
try:
        conn = psycopg2.connect(**db_params,database=new_database)
        conn.autocommit = True
        cursor = conn.cursor()
        cursor.execute("UPDATE suppliers_send_setting SET mail_1 = 'epsminhtrihue@gmail.com', mail_2 = NULL, mail_3 = NULL, mail_4 = NULL, mail_5 = NULL")

except Exception as e:
        print(f'Error: {e}')